In [2]:
import pandas as pd
import numpy as np

In [55]:
df_cg1d = pd.read_csv('beam_flux_cg1d.txt', sep='\t', header=None, skiprows=1)
df_venus = pd.read_csv('beam_flux_venus.txt', sep='\t', header=None, skiprows=1)
df_snap = pd.read_csv('beam_flux_snap.txt', sep='\t', header=None, skiprows=1)

In [56]:
df_dict = {
    'cg1d': df_cg1d,
    'venus': df_venus,
    'snap': df_snap
}

In [57]:
for each in df_dict.keys():
    df_dict[each] = df_dict[each][df_dict[each][0]<=max(df_dict['cg1d'][0])]
    df_dict[each] = df_dict[each][df_dict[each][0]>=min(df_dict['cg1d'][0])]
    print(df_dict[each].head())

          0             1
0  0.812655  1.776169e+04
1  1.028404  1.201199e+05
2  1.244153  2.919421e+05
3  1.459902  5.860009e+05
4  1.675651  1.030660e+06
        0          1
33  0.859  8988576.0
34  0.908  8638348.8
35  0.964  8301196.8
36  1.015  8126496.0
37  1.069  7963622.4
         0           1
139  0.854  16600000.0
140  0.904  15700000.0
141  0.958  15000000.0
142  1.010  14400000.0
143  1.070  13900000.0


In [58]:
for each in df_dict.keys():
    integr = np.trapz(y=df_dict[each][1], x=df_dict[each][0], dx=0.01).round(3)
    print(each+': ', integr, '(n/s/cm^2)')

cg1d:  6512271.475 (n/s/cm^2)
venus:  35914705.214 (n/s/cm^2)
snap:  67210590.0 (n/s/cm^2)
